In [7]:
# Load dataset
path_to_datafile = 'data/100134_1.8MW_55s_aaro_5_3.npz'
data_files = np.load(path_to_datafile, allow_pickle=True)
data = {key: data_files[key] for key in data_files.files}

# Unpack
prob = data['prob'][()]
x = prob['x']
t = prob['tout']
prob = data['prob'][()]
x = prob['x']                        # Normalized poloidal flux
D = prob['D']                        # Diffusion coefficient
V = prob['V']                        # Pinch velocity
S = prob['S']                        # Source term
dVdx = prob['dVdx']                  # Volume expansion term
g1 = prob['g1']                      # ⟨∇ρ⟩
g2 = prob['g2']                      # ⟨(∇ρ)^2⟩
bc = prob['bc']                      # Boundary condition at psi_n ≈ 1.106 (or 1.0 ???)
n0 = prob['n0']                      # Initial density
t = prob['tout']                     # Time
itaupar = prob['itaupar']            # Parallel loss time

In [ ]:
rho = x
dV_drho = dVdx

itau_parallel = itaupar[:, 0]

(105,)

In [17]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Define constants and parameters
tau_parallel = 1e3  # Example value, adjust as needed

# Define initial guesses for D, V, and S
D0 = lambda rho: 1 + rho  # Example function, adjust as needed
D1 = lambda rho: 0.1 + 0.05 * rho
V0 = lambda rho: -0.5 * rho
V1 = lambda rho: 0.1 - 0.05 * rho
S0 = 1.0
S1 = 0.5
omega = 1.0  # Example frequency, adjust as needed
phi_D = lambda rho: np.pi * rho
phi_V = lambda rho: np.pi * (1 - rho)
phi_S = 0.0

# Define the electron density evolution equation with improved numerical stability
def electron_density_evolution(ne, D, V, S, rho):
    term1 = np.gradient(dV_drho * (g2 * D(rho) * np.gradient(ne, rho)) - g1 * V(rho) * ne, rho)
    source_term = dV_drho*(S(rho) - ne * itau_parallel)
    return np.nan_to_num(term1 + source_term)  # Replace NaNs with zero

# Objective function to minimize
def objective(params):
    D = lambda rho: params[0] + params[1] * np.sin(omega * rho - params[2] * rho)
    V = lambda rho: params[3] + params[4] * np.sin(omega * rho - params[5] * rho)
    S = lambda rho: params[6] + params[7] * np.sin(omega * rho - params[8])
    ne = np.ones_like(rho)
    
    # Solve the steady-state electron density
    for i in range(1000):  # Iterate to steady state
        ne_new = electron_density_evolution(ne, D, V, S, rho)
        if np.max(np.abs(ne_new - ne)) < 1e-4:
            break
        ne = ne_new
    
    return np.sum((ne - target_ne)**2)

# Initial parameter guesses
initial_params = [1.0, 0.1, 0.1, -0.5, 0.1, 0.1, 1.0, 0.5, 0.0]

# Target electron density profile for optimization (example)
target_ne = np.linspace(1, 0.1, len(rho))

# Optimize the parameters
result = minimize(objective, initial_params, method='Nelder-Mead')
optimal_params = result.x

# Print the optimal parameters
print("Optimal Parameters:", optimal_params)

# Define the optimal functions for D, V, and S using the optimized parameters
D_opt = lambda rho: optimal_params[0] + optimal_params[1] * np.sin(omega * rho - optimal_params[2] * rho)
V_opt = lambda rho: optimal_params[3] + optimal_params[4] * np.sin(omega * rho - optimal_params[5] * rho)
S_opt = lambda rho: optimal_params[6] + optimal_params[7] * np.sin(omega * rho - optimal_params[8])

# Solve for the steady-state electron density with the optimal parameters
ne_opt = np.ones_like(rho)
for i in range(1000):
    ne_new = electron_density_evolution(ne_opt, D_opt, V_opt, S_opt, rho, tau_parallel)
    if np.max(np.abs(ne_new - ne_opt)) < 1e-4:
        break
    ne_opt = ne_new

# Plot the results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(rho, D_opt(rho))
plt.title("Optimized Diffusion Coefficient D(rho)")
plt.xlabel("rho")
plt.ylabel("D(rho)")

plt.subplot(2, 2, 2)
plt.plot(rho, V_opt(rho))
plt.title("Optimized Convection Velocity V(rho)")
plt.xlabel("rho")
plt.ylabel("V(rho)")

plt.subplot(2, 2, 3)
plt.plot(rho, S_opt(rho))
plt.title("Optimized Particle Source S(rho)")
plt.xlabel("rho")
plt.ylabel("S(rho)")

plt.subplot(2, 2, 4)
plt.plot(rho, ne_opt, label="Optimized n_e(rho)")
plt.plot(rho, target_ne, label="Target n_e(rho)", linestyle='--')
plt.title("Electron Density Profile")
plt.xlabel("rho")
plt.ylabel("n_e(rho)")
plt.legend()

plt.tight_layout()
plt.show()

C:\Users\jonas\AppData\Local\Temp\ipykernel_8100\848049700.py:22: RuntimeWarning: overflow encountered in multiply
  term1 = np.gradient(dV_drho * (g2 * D(rho) * np.gradient(ne, rho)) - g1 * V(rho) * ne, rho)
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\function_base.py:1250: RuntimeWarning: overflow encountered in multiply
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\function_base.py:1250: RuntimeWarning: invalid value encountered in multiply
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\function_base.p

KeyboardInterrupt: 